In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.vision.gan import *
from ipyexperiments import *
exp1 = IPyExperimentsPytorch()

In [ ]:
path = Path("celeba/img_align_celeba")

In [ ]:
def get_data(bs, size):
    return (GANItemList.from_folder(path, noise_sz=100)
               .split_none()
               .label_from_func(noop)
               .transform(tfms=[[crop_pad(size=size, row_pct=(0,1), col_pct=(0,1))], []], size=size, tfm_y=True)
               .databunch(bs=bs)
               .normalize(stats = [torch.tensor([0.5,0.5,0.5]), torch.tensor([0.5,0.5,0.5])], do_x=False, do_y=True))

In [ ]:
data = get_data(128, 64)

In [ ]:
data.show_batch(rows=5)

In [ ]:
generator = basic_generator(in_size=64, n_channels=3, n_extra_layers=3, leaky=0)
critic    = basic_critic   (in_size=64, n_channels=3, n_extra_layers=3, transpose=False)

In [ ]:
# learn = GANLearner.wgan(data, generator, critic, switch_eval=False,
#                         opt_func = partial(optim.Adam, betas = (0.5,0.99)), wd=0., gen_loss_func=nn.BCELoss() ,crit_loss_func=nn.BCELoss())

In [ ]:
def gan_loss_from_func(loss_gen, loss_crit, weights_gen:Tuple[float,float]=None):
    "Define loss functions for a GAN from `loss_gen` and `loss_crit`."
    def _loss_G(fake_pred, output, target, weights_gen=weights_gen):
        ones = fake_pred.new_ones(fake_pred.shape[0])
        weights_gen = ifnone(weights_gen, (1.,1.))
        return weights_gen[0] * loss_crit(fake_pred, ones) + weights_gen[1] * loss_gen(output, target)

    def _loss_C(real_pred, fake_pred):
        ones  = real_pred.new_ones (real_pred.shape[0])
        zeros = fake_pred.new_zeros(fake_pred.shape[0])
        return (loss_crit(real_pred, ones) + loss_crit(fake_pred, zeros)) / 2

    return _loss_G, _loss_C

In [ ]:
losses = gan_loss_from_func(nn.BCELoss(), nn.BCELoss(), weights_gen=(0,0.02))
learn = GANLearner(data, generator, critic, ,switch_eval=False, opt_func = partial(optim.Adam, betas = (0.5,0.99)))

In [ ]:


learn.fit(5,2e-4)

